In [11]:
import os
cwd = os.getcwd()
cwd

'/Users/jcelik/Coding/ik2200/cassandra-strict-slo/data_analysis'

In [12]:
RESULTS_DIR = "../results"

In [13]:
import os
for file in os.listdir(RESULTS_DIR):
    if file.startswith("mixload_1000000ops") and file.endswith("Client1"):
        print(file)

mixload_1000000ops_10_Client1
mixload_1000000ops_1_Client1
mixload_1000000ops_0_Client1
mixload_1000000ops_30_Client1
mixload_1000000ops_20_Client1
mixload_1000000ops_5_Client1
mixload_1000000ops_40_Client1


In [33]:
mixload_files = list(map(lambda file: RESULTS_DIR + "/" + file,
                    filter(lambda file: 
                       file.startswith("mixload_1000000ops") and file.endswith("Client1"),
                       os.listdir(RESULTS_DIR)
                    )))
mixload_files

['../results/mixload_1000000ops_10_Client1',
 '../results/mixload_1000000ops_1_Client1',
 '../results/mixload_1000000ops_0_Client1',
 '../results/mixload_1000000ops_30_Client1',
 '../results/mixload_1000000ops_20_Client1',
 '../results/mixload_1000000ops_5_Client1',
 '../results/mixload_1000000ops_40_Client1']

In [62]:
def get_percentiles(filename):
    file = open(filename, "r")
    lines = list(map(lambda line: line.rstrip('\n'), file.readlines()))

    # Remove everything before "Starting test"
    def get_line_with_find(lines, regex):
        for index, line in enumerate(lines):
            if line.find(regex) != -1:
                return index
        return -1

    test_index = get_line_with_find(lines, 'Starting test.')

    full_test = lines[test_index + 1:]
    
    read_test_lines = list(
        filter(lambda line: line.find("[READ]") != -1, 
        filter(lambda line: line.find("Percentile") != -1, full_test)
    ))

    def get_tuple_from_line(line):
        parts = line.split(",")
        
        percentile_string = parts[1].strip()
        percentile = percentile_string.split("Percentile")[0].replace("th", "")
        percentile = float(percentile)
        
        latency_string = parts[2].strip()
        latency = int(latency_string)
        
        return percentile, latency
    
    return list(map(get_tuple_from_line, read_test_lines))
    
#     print(get_tuple_from_line(read_test_lines[0]))
#     for line in read_test_lines:
#         print(line)
    
    
get_percentiles(mixload_files[0])

[(10.0, 1430),
 (20.0, 1817),
 (30.0, 2191),
 (40.0, 2605),
 (50.0, 3031),
 (60.0, 3483),
 (70.0, 3977),
 (80.0, 4683),
 (90.0, 6007),
 (95.0, 7843),
 (96.0, 8607),
 (97.0, 9583),
 (98.0, 11071),
 (99.0, 13823),
 (99.1, 14327),
 (99.2, 14919),
 (99.3, 15655),
 (99.4, 16495),
 (99.5, 17503),
 (99.6, 18639),
 (99.7, 20463),
 (99.8, 23311),
 (99.85, 25135),
 (99.89, 27599),
 (99.9, 28495)]

In [70]:
def get_cdf_for_file(filename):
    threshold = filename.split("1000000ops_")[1].split("_")[0]
    return threshold, get_percentiles(filename)
    
get_cdf_for_file(mixload_files[0])

('10',
 [(10.0, 1430),
  (20.0, 1817),
  (30.0, 2191),
  (40.0, 2605),
  (50.0, 3031),
  (60.0, 3483),
  (70.0, 3977),
  (80.0, 4683),
  (90.0, 6007),
  (95.0, 7843),
  (96.0, 8607),
  (97.0, 9583),
  (98.0, 11071),
  (99.0, 13823),
  (99.1, 14327),
  (99.2, 14919),
  (99.3, 15655),
  (99.4, 16495),
  (99.5, 17503),
  (99.6, 18639),
  (99.7, 20463),
  (99.8, 23311),
  (99.85, 25135),
  (99.89, 27599),
  (99.9, 28495)])

In [72]:
cdfs = list(map(get_cdf_for_file, mixload_files))
cdfs[0]

('10',
 [(10.0, 1430),
  (20.0, 1817),
  (30.0, 2191),
  (40.0, 2605),
  (50.0, 3031),
  (60.0, 3483),
  (70.0, 3977),
  (80.0, 4683),
  (90.0, 6007),
  (95.0, 7843),
  (96.0, 8607),
  (97.0, 9583),
  (98.0, 11071),
  (99.0, 13823),
  (99.1, 14327),
  (99.2, 14919),
  (99.3, 15655),
  (99.4, 16495),
  (99.5, 17503),
  (99.6, 18639),
  (99.7, 20463),
  (99.8, 23311),
  (99.85, 25135),
  (99.89, 27599),
  (99.9, 28495)])

In [ ]:
# Someone plot cdfs here